In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import sys
import os
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kamakshibansal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kamakshibansal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os
import glob
import pandas as pd
os.chdir("/Users/kamakshibansal/Moorfields/Letters/File_with_1_diagnosis")

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [4]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
# combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [5]:
combined_csv.shape

(1293790, 4)

In [6]:
0.8*275238

220190.40000000002

In [7]:
combined_csv.head()

,Unnamed: 0,LetterBody,final_sentence,Diagnosis2
0,4,urgent care clinic \r\n\r\n- diagnosis: haemor...,urgent care clinic diagnosis haemorrhagic post...,posterior vitreous detachment
1,8,urgent care clinic\r\n\r\n- diagnosis: healing...,urgent care clinic diagnosis corneal abrasion ...,corneal abrasion
2,12,urgent eye care\r\n\r\n- diagnosis: acute ante...,urgent eye care diagnosis acute anterior uveit...,acute anterior uveitis
3,15,attended urgent eye care clinic with her son\r...,attended urgent eye care clinic her son diagno...,corneal abrasion
4,16,you have kindly reviewed this 73 year old lady...,you have kindly reviewed this 73 year old lady...,cataract


In [8]:

# combined_csv['split'] = np.random.randn(combined_csv.shape[0], 1)

msk = np.random.rand(len(combined_csv)) <= 0.8

train = combined_csv[msk]
test = combined_csv[~msk]

In [9]:
train.shape

(1034798, 4)

In [10]:
test.shape

(258992, 4)

In [11]:
train.head()

,Unnamed: 0,LetterBody,final_sentence,Diagnosis2
0,4,urgent care clinic \r\n\r\n- diagnosis: haemor...,urgent care clinic diagnosis haemorrhagic post...,posterior vitreous detachment
1,8,urgent care clinic\r\n\r\n- diagnosis: healing...,urgent care clinic diagnosis corneal abrasion ...,corneal abrasion
2,12,urgent eye care\r\n\r\n- diagnosis: acute ante...,urgent eye care diagnosis acute anterior uveit...,acute anterior uveitis
3,15,attended urgent eye care clinic with her son\r...,attended urgent eye care clinic her son diagno...,corneal abrasion
4,16,you have kindly reviewed this 73 year old lady...,you have kindly reviewed this 73 year old lady...,cataract


In [12]:
test.head()

,Unnamed: 0,LetterBody,final_sentence,Diagnosis2
8,31,this patient attended for a routine follow-up ...,this patient attended for routine follow-up ap...,glaucoma
10,36,"- diagnosis: bilateral open-angle glaucoma, st...",diagnosis bilateral open-angle glaucoma steroi...,glaucoma
20,56,this 76 year old man attended the clinic for a...,this 76 year old man attended clinic for routi...,cataract
24,69,this chap presented to a&e with 6 months histo...,this presented to a&e 6 months history waterin...,conjunctivitis
27,72,this 80 year old attended the glaucoma clinic...,this 80 year old attended glaucoma clinic for ...,glaucoma


In [13]:
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer


stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # remove non letters
    text = re.sub("[^a-zA-Z]", " ", text)
    # tokenize
    tokens = nltk.word_tokenize(text)
    # stem
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = 'english',
    max_features = 150
)

In [14]:
corpus_data_features = vectorizer.fit_transform(train.LetterBody.tolist() + test.LetterBody.tolist())

/Users/kamakshibansal/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not i

In [15]:
corpus_data_features_nd = corpus_data_features.toarray()
corpus_data_features_nd.shape

(1293790, 150)

In [16]:
vocab = vectorizer.get_feature_names()
print (vocab)

['acuiti', 'advis', 'angl', 'ani', 'ansi', 'ansicpg', 'anterior', 'appoint', 'arial', 'arrang', 'attend', 'b', 'bd', 'benefit', 'bilater', 'blood', 'blue', 'care', 'cataract', 'cc', 'cf', 'chang', 'clinic', 'colortbl', 'consult', 'continu', 'control', 'correspond', 'current', 'day', 'dear', 'deff', 'deflang', 'diabet', 'diagnosi', 'disc', 'discuss', 'dob', 'dr', 'drop', 'e', 'evid', 'examin', 'eye', 'f', 'fcharset', 'fellow', 'field', 'fnil', 'follow', 'fonttbl', 'fprq', 'frcophth', 'fs', 'fswiss', 'g', 'glass', 'glaucoma', 'gp', 'grate', 'green', 'ha', 'hi', 'hospit', 'improv', 'intraocular', 'iol', 'iop', 'l', 'ladi', 'lang', 'laser', 'le', 'left', 'list', 'london', 'macular', 'man', 'manag', 'medic', 'mg', 'mild', 'miss', 'mmhg', 'month', 'moorfield', 'mr', 'ms', 'n', 'nh', 'noct', 'normal', 'nsc', 'number', 'ocular', 'od', 'oedema', 'old', 'oper', 'optic', 'origin', 'par', 'pard', 'patient', 'pinhol', 'pleas', 'prescrib', 'preserv', 'pressur', 'primari', 'r', 'recipi', 'red', 'refe

In [17]:
from sklearn.model_selection import train_test_split

# remember that corpus_data_features_nd contains all of our original train and test data, so we need to exclude
# the unlabeled test entries
X_train, X_test, y_train, y_test  = train_test_split(
    corpus_data_features_nd[0:len(train)], 
    train.Diagnosis2,
    train_size=0.85, 
    random_state=1234)

In [18]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)

/Users/kamakshibansal/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/kamakshibansal/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/kamakshibansal/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [19]:
y_pred = log_model.predict(X_test)

In [20]:
y_pred

array(['cataract', 'glaucoma', 'glaucoma', ..., 'glaucoma', 'cataract',
       'cataract'], dtype=object)

In [21]:
# from sklearn.model_selection import cross_validate
# from sklearn import metrics, cross_validation
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

metrics.accuracy_score(y_test, y_pred)

0.9432547352145342